# Link budgets

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ephemerista
from ephemerista.analysis.link_budget import LinkBudget
from ephemerista.angles import Angle
from ephemerista.assets import Asset, GroundStation, Spacecraft
from ephemerista.comms.antennas import SimpleAntenna
from ephemerista.comms.channels import Channel
from ephemerista.comms.frequencies import Frequency
from ephemerista.comms.receiver import SimpleReceiver
from ephemerista.comms.systems import CommunicationSystem
from ephemerista.comms.transmitter import Transmitter
from ephemerista.propagators.sgp4 import SGP4
from ephemerista.scenarios import Scenario
from ephemerista.time import TimeDelta

In [ ]:
ephemerista.init(eop_path="../tests/resources/finals2000A.all.csv", spk_path="../tests/resources/de440s.bsp")

## Link budget without interference

In [ ]:
uplink = Channel(link_type="uplink", modulation="BPSK", data_rate=430502, required_eb_n0=2.3, margin=3)
downlink = Channel(link_type="downlink", modulation="BPSK", data_rate=861004, required_eb_n0=4.2, margin=3)

In [ ]:
# S-Band
frequency = Frequency.megahertz(8308)

In [ ]:
gs_antenna = SimpleAntenna(gain_db=30, beamwidth_deg=5, design_frequency=frequency)
gs_transmitter = Transmitter(power=4, frequency=frequency, line_loss=1.0)
gs_receiver = SimpleReceiver(system_noise_temperature=889, frequency=frequency)
gs_system = CommunicationSystem(
    channels=[uplink.channel_id, downlink.channel_id],
    transmitter=gs_transmitter,
    receiver=gs_receiver,
    antenna=gs_antenna,
)

In [ ]:
sc_antenna = SimpleAntenna(gain_db=6.5, beamwidth_deg=60, design_frequency=frequency)
sc_transmitter = Transmitter(power=1.348, frequency=frequency, line_loss=1.0)
sc_receiver = SimpleReceiver(system_noise_temperature=429, frequency=frequency)
sc_system = CommunicationSystem(
    channels=[uplink.channel_id, downlink.channel_id],
    transmitter=sc_transmitter,
    receiver=sc_receiver,
    antenna=sc_antenna,
)

In [ ]:
station_coordinates = [
    (38.017, 23.731),
    (36.971, 22.141),
    (39.326, -82.101),
    (50.750, 6.211),
]

stations = [
    Asset(
        model=GroundStation.from_lla(longitude, latitude, minimum_elevation=Angle.from_degrees(10)),
        name=f"Station {i}",
        comms=[gs_system],
    )
    for i, (latitude, longitude) in enumerate(station_coordinates)
]

In [ ]:
tle1 = """
1 99878U 14900A   24103.76319466  .00000000  00000-0 -11394-2 0    01
2 99878  97.5138 156.7457 0016734 205.2381 161.2435 15.13998005    06
"""

propagator1 = SGP4(tle=tle1)
sc1 = Asset(model=Spacecraft(propagator=propagator1), name="PHASMA", comms=[sc_system])

Set all ground stations to track the spacecraft, and set the spacecraft to track the first ground station.

Note: if no tracking is specified, the antenna's boresight vector is used for computing pattern angle losses (in LVLH frame for a spacecraft, and in topocentric frame for a ground station).

In [ ]:
for station in stations:
    station.track(sc1.asset_id)

sc1.track(stations[0].asset_id)

In [ ]:
start_time = propagator1.time
end_time = start_time + TimeDelta.from_days(1)

scenario1 = Scenario(
    assets=[*stations, sc1],
    channels=[uplink, downlink],
    name="PHASMA Link Budget",
    start_time=start_time,
    end_time=end_time,
)

Showing the overview of the ground station passes as a dataframe.

In [ ]:
lb = LinkBudget(scenario=scenario1)
results = lb.analyze()
results.to_dataframe(stations[0], sc1)

Showing the detailed metrics of a single pass as a dataframe

In [ ]:
results[stations[0], sc1][0].to_dataframe()

Showing plots between the first ground station and the spacecraft. As both are tracking each other, both the TX and the RX angles are always 0.

In [ ]:
results[stations[0], sc1][0].plot()

Showing a similar plot but between the second ground station and the spacecraft. And the spacecraft is not tracking this ground station but another one nearby, the TX angle is not zero but relatively low.

In [ ]:
results[stations[1], sc1][0].plot()

Plotting the environmental attenuations

In [ ]:
results[stations[0], sc1][0].plot_attenuations(lb.percentage_exceed)

## Link budget with interference analysis

Adding a nearby spacecraft to the scenario to demonstrate downlink interference. 

As the scenario already contains several ground stations close to each other, there will be uplink interference too.

In [ ]:
tle2 = """
1 99878U 14900A   24103.76319466  .00000000  00000-0 -11394-2 0    01
2 99878  97.5138 156.7457 0016734 205.2381 191.2435 15.13998005    09
"""

propagator2 = SGP4(tle=tle2)
sc2 = Asset(model=Spacecraft(propagator=propagator1), name="INTERFERER", comms=[sc_system])

scenario2 = Scenario(
    assets=[*stations, sc1, sc2],
    channels=[uplink, downlink],
    name="PHASMA Link Budget with interference",
    start_time=start_time,
    end_time=end_time,
)

lb2 = LinkBudget(scenario=scenario2, with_interference=True)
results_with_interference = lb2.analyze()

In [ ]:
results_with_interference.to_dataframe(stations[0], sc1, with_interference=True)

Normal link budget without interference

In [ ]:
idx_pass = 1

In [ ]:
results_with_interference[stations[0], sc1][idx_pass].plot(plot_interference=False)

Same link budget but with interference

In [ ]:
results_with_interference[stations[0], sc1][idx_pass].to_dataframe()

In [ ]:
results_with_interference[stations[0], sc1][idx_pass].plot(plot_interference=True)